# preprocess essential climate variables

In [1]:
import os
import numpy as np
import pandas as pd
import netCDF4 as nc
import xarray as xr
import geopandas as gpd

In [2]:
def select_bounding_box_region(path, years, ecv, lon_min, lon_max, lat_min, lat_max):
    
    data_final = pd.DataFrame(columns=['lat', 'lon', 'time', ecv])
    
    for year in years:
        print('Processing {}...'.format(year))
        files = os.listdir(path+str(year))
        
        for file in files:
            data = xr.open_dataset(path+str(year)+'/'+file)
            data = data.to_dataframe().reset_index()
            data_filtered = data[(data.lon <= lon_max) & (data.lon >= lon_min) & (data.lat >= lat_min) & (data.lat <= lat_max)].dropna(subset=[ecv])[['lat', 'lon', 'time', ecv]]
            data_final = data_final.append(data_filtered)
            
    data_final = data_final.reset_index(drop=True)
    
    return data_final

In [3]:
years = [2010,
         2011,
         2012,
         2013,
         2014,
         2015]

In [4]:
lon_min, lon_max, lat_min, lat_max = 60, 100, 0, 40

## sea surface salinity

In [5]:
!bash download_sea_surface_salinity_data.sh

Download sea surface salinity data...
--2021-02-19 18:27:10--  ftp://anon-ftp.ceda.ac.uk/neodc/esacci/sea_surface_salinity/data/v02.31/30days/2010
           => ‘../data/sea_surface_salinity/2010/.listing’
Resolving anon-ftp.ceda.ac.uk (anon-ftp.ceda.ac.uk)... 130.246.130.100
Connecting to anon-ftp.ceda.ac.uk (anon-ftp.ceda.ac.uk)|130.246.130.100|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /neodc/esacci/sea_surface_salinity/data/v02.31/30days ... done.
==> PASV ... done.    ==> LIST ... done.

.listing                [ <=>                ]     818  --.-KB/s    in 0,02s   

2021-02-19 18:27:11 (45,8 KB/s) - ‘../data/sea_surface_salinity/2010/.listing’ saved [818]

Removed ‘../data/sea_surface_salinity/2010/.listing’.
--2021-02-19 18:27:11--  ftp://anon-ftp.ceda.ac.uk/neodc/esacci/sea_surface_salinity/data/v02.31/30days/2010/2010
           => ‘../data/sea_surface_salinity/2010/.listing’
==> CWD (1) /n

In [6]:
path = '../data/sea_surface_salinity/'

In [7]:
data = nc.Dataset(path+str(years[0])+'/'+sorted(os.listdir(path+str(years[0])))[0])

In [8]:
for var in data.variables.values():
    print(var)

<class 'netCDF4._netCDF4.Variable'>
int16 noutliers(time, lat, lon)
    long_name: Count of the Number of Outliers within this bin cell
    _FillValue: -1
    valid_max: 10000
    valid_min: 0
unlimited dimensions: time
current shape = (1, 584, 1388)
filling on
<class 'netCDF4._netCDF4.Variable'>
int16 total_nobs(time, lat, lon)
    long_name: Number of SSS in the time interval
    _FillValue: -1
    valid_max: 10000
    valid_min: 0
unlimited dimensions: time
current shape = (1, 584, 1388)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 sss_bias(time, lat, lon)
    _FillValue: nan
    long_name: Bias in Sea Surface Salinity
    valid_max: 100.0
    valid_min: -100.0
unlimited dimensions: time
current shape = (1, 584, 1388)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 sss_random_error(time, lat, lon)
    _FillValue: nan
    long_name: Sea Surface Salinity Random Error
    valid_max: 100.0
    valid_min: 0.0
unlimited dimensions: time
current shape = (1, 584, 1388)
fill

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  


In [9]:
%%time

sss = select_bounding_box_region(path, years, 'sss', lon_min, lon_max, lat_min, lat_max)

Processing 2010...
Processing 2011...
Processing 2012...
Processing 2013...
Processing 2014...
Processing 2015...
CPU times: user 29.5 s, sys: 2.87 s, total: 32.4 s
Wall time: 32.9 s


In [10]:
sss.shape

(1817422, 4)

In [11]:
sss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1817422 entries, 0 to 1817421
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   lat     float64       
 1   lon     float64       
 2   time    datetime64[ns]
 3   sss     float32       
dtypes: datetime64[ns](1), float32(1), float64(2)
memory usage: 48.5 MB


In [12]:
sss.head()

,lat,lon,time,sss
0,0.098082,60.043228,2010-04-15,35.378178
1,0.098082,60.302593,2010-04-15,35.311237
2,0.098082,60.561958,2010-04-15,35.226299
3,0.098082,60.821327,2010-04-15,35.049088
4,0.098082,61.080692,2010-04-15,35.022762


In [13]:
sss['year'] = sss.time.apply(lambda x: x.year)

In [14]:
sss['month'] = sss.time.apply(lambda x: x.month)

In [15]:
sss = sss[['lat', 'lon', 'year', 'month', 'sss']].groupby(['lat', 'lon', 'year', 'month']).aggregate('mean').reset_index()

In [16]:
sss = gpd.GeoDataFrame(sss, geometry=gpd.points_from_xy(sss.lon, sss.lat), crs={'init': 'epsg:4326'})

In [17]:
sss = sss.drop(['lat', 'lon'], axis=1)

In [18]:
sss.shape

(908712, 4)

In [19]:
sss.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 908712 entries, 0 to 908711
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   year      908712 non-null  int64   
 1   month     908712 non-null  int64   
 2   sss       908712 non-null  float32 
 3   geometry  908712 non-null  geometry
dtypes: float32(1), geometry(1), int64(2)
memory usage: 24.3 MB


In [20]:
sss.head()

,year,month,sss,geometry
0,2010,1,35.265869,POINT (60.04323 0.09808)
1,2010,2,35.191532,POINT (60.04323 0.09808)
2,2010,3,35.426231,POINT (60.04323 0.09808)
3,2010,4,35.370430,POINT (60.04323 0.09808)
4,2010,5,35.310860,POINT (60.04323 0.09808)


In [21]:
sss.to_file(path+'monthly_sss_2010_2015.shp')

## chlorophyll-a concentration

In [22]:
!bash download_chlorophyll_a_concentration_data.sh

Download chlorophyll-a concentration data...
--2021-02-19 18:29:55--  ftp://anon-ftp.ceda.ac.uk/neodc/esacci/ocean_colour/data/v4.2-release/geographic/netcdf/chlor_a/monthly/v4.2/2010
           => ‘../data/chlorophyll_a_concentration/2010/.listing’
Resolving anon-ftp.ceda.ac.uk (anon-ftp.ceda.ac.uk)... 130.246.130.100
Connecting to anon-ftp.ceda.ac.uk (anon-ftp.ceda.ac.uk)|130.246.130.100|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /neodc/esacci/ocean_colour/data/v4.2-release/geographic/netcdf/chlor_a/monthly/v4.2 ... done.
==> PASV ... done.    ==> LIST ... done.

.listing                [ <=>                ]   1,48K  --.-KB/s    in 0,03s   

2021-02-19 18:29:56 (58,6 KB/s) - ‘../data/chlorophyll_a_concentration/2010/.listing’ saved [1520]

Removed ‘../data/chlorophyll_a_concentration/2010/.listing’.
--2021-02-19 18:29:56--  ftp://anon-ftp.ceda.ac.uk/neodc/esacci/ocean_colour/data/v4.2-release/geo

In [23]:
path = '../data/chlorophyll_a_concentration/'

In [24]:
data = nc.Dataset(path+str(years[0])+'/'+sorted(os.listdir(path+str(years[0])))[0])

In [25]:
for var in data.variables.values():
    print(var)

<class 'netCDF4._netCDF4.Variable'>
float32 MERIS_nobs_sum(time, lat, lon)
    _FillValue: 0.0
    long_name: Count of the number of observations from the MERIS sensor contributing to this bin cell
    number_of_files_composited: 31
unlimited dimensions: 
current shape = (1, 4320, 8640)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 MODISA_nobs_sum(time, lat, lon)
    _FillValue: 0.0
    long_name: Count of the number of observations from the MODIS (Aqua) sensor contributing to this bin cell
    number_of_files_composited: 31
unlimited dimensions: 
current shape = (1, 4320, 8640)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 SeaWiFS_nobs_sum(time, lat, lon)
    _FillValue: 0.0
    long_name: Count of the number of observations from the SeaWiFS (GAC and LAC) sensor contributing to this bin cell
    number_of_files_composited: 31
unlimited dimensions: 
current shape = (1, 4320, 8640)
filling on
<class 'netCDF4._netCDF4.Variable'>
float32 VIIRS_nobs_sum(time, lat, lon)
  

/home/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  


In [26]:
%%time

chlor_a = select_bounding_box_region(path, years, 'chlor_a', lon_min, lon_max, lat_min, lat_max)

Processing 2010...
Processing 2011...
Processing 2012...
Processing 2013...
Processing 2014...
Processing 2015...
CPU times: user 8min 20s, sys: 2min 34s, total: 10min 54s
Wall time: 10min 34s


In [27]:
chlor_a.shape

(26001084, 4)

In [28]:
chlor_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26001084 entries, 0 to 26001083
Data columns (total 4 columns):
 #   Column   Dtype         
---  ------   -----         
 0   lat      float64       
 1   lon      float64       
 2   time     datetime64[ns]
 3   chlor_a  float32       
dtypes: datetime64[ns](1), float32(1), float64(2)
memory usage: 694.3 MB


In [29]:
chlor_a.head()

,lat,lon,time,chlor_a
0,39.895832,64.770836,2010-04-01,8.540924
1,39.895832,64.812500,2010-04-01,8.540924
2,39.895832,64.854164,2010-04-01,9.433975
3,39.854168,64.729164,2010-04-01,8.993294
4,39.854168,64.770836,2010-04-01,8.993294


In [30]:
chlor_a['year'] = chlor_a.time.apply(lambda x: x.year)

In [31]:
chlor_a['month'] = chlor_a.time.apply(lambda x: x.month)

In [32]:
chlor_a['chlor_a_log'] = np.log(chlor_a['chlor_a'])

In [33]:
chlor_a = chlor_a.drop(['time', 'chlor_a'], axis=1)

In [34]:
chlor_a = gpd.GeoDataFrame(chlor_a, geometry=gpd.points_from_xy(chlor_a.lon, chlor_a.lat), crs={'init': 'epsg:4326'})

In [35]:
chlor_a = chlor_a.drop(['lat', 'lon'], axis=1)

In [36]:
chlor_a.shape

(26001084, 4)

In [37]:
chlor_a.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 26001084 entries, 0 to 26001083
Data columns (total 4 columns):
 #   Column       Dtype   
---  ------       -----   
 0   year         int64   
 1   month        int64   
 2   chlor_a_log  float32 
 3   geometry     geometry
dtypes: float32(1), geometry(1), int64(2)
memory usage: 694.3 MB


In [38]:
chlor_a.head()

,year,month,chlor_a_log,geometry
0,2010,4,2.144869,POINT (64.77084 39.89583)
1,2010,4,2.144869,POINT (64.81250 39.89583)
2,2010,4,2.244318,POINT (64.85416 39.89583)
3,2010,4,2.196479,POINT (64.72916 39.85417)
4,2010,4,2.196479,POINT (64.77084 39.85417)


In [39]:
for year in years:
    chlor_a[chlor_a.year == year].to_file(path+'monthly_chlor_a_'+str(year)+'.shp')